In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')

import requests
from bs4 import BeautifulSoup

# Función para scrapear los datos de la tabla y agregar el tier correspondiente
def scrape_tier_data(tier):
    # URL de la página web con el parámetro skillTier
    url = f'https://www.overbuff.com/rankings?platform=pc&gameMode=competitive&hero=ana&season=ow2s11&skillTier={tier}'

    # Realizar una solicitud GET a la URL
    response = requests.get(url, headers = {'User-agent': 'your bot 0.1'})

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', class_='w-full whitespace-nowrap')
    rows = table.find_all('tr')
    data = []

    for row in rows:
      # Encuentra todos los 'td' de la fila
      cells = row.find_all('td')
      if len(cells) > 1:  # Asegúrate de que haya al menos dos columnas
          # Obtén el texto del segundo 'td'
          player_with_tier = cells[1].get_text(strip=True)
          # Capitaliza el 'tier' y ajusta el nombre del jugador
          capitalized_tier = tier.capitalize()
          if player_with_tier.endswith(capitalized_tier):
              player_name = player_with_tier[:-len(capitalized_tier)].strip()
          else:
              player_name = player_with_tier
          # Reemplaza el nombre del jugador en la lista y añade el tier
          row_data = [player_name, capitalized_tier]
          data.append(row_data)
    return data

def get_player_data(player):
  intentos=0
  continuar=True
  nick, number = player.split('#')
  data=None
  while intentos < 3 and continuar :
    api_url = f'https://overfast-api.tekrop.fr/players/{nick}-{number}'
    response = requests.get(api_url, headers = {'User-agent': 'your bot 0.1'})
    if response.status_code == 200:
        intentos=4
        continuar=False
        try:
            data = response.json()
        except ValueError:
            print(f"No se pudo decodificar el JSON para el jugador {player}.")
    else:
        intentos=intentos+1
        print(f"Error al obtener datos para {player}: {response.status_code}.")
  return data

# Función para realizar la solicitud a la API y extraer los datos de Ana
def get_player_stats(player):
    data = get_player_data(player)
    campos_cero=0
    if data:

      # Campos de interés en heroes_comparisons
      attributes = [
          'time_played', 'games_won', 'win_percentage', 'weapon_accuracy_best_in_game',
          'eliminations_per_life', 'kill_streak_best', 'eliminations_avg_per_10_min',
          'deaths_avg_per_10_min', 'final_blows_avg_per_10_min', 'solo_kills_avg_per_10_min',
          'objective_kills_avg_per_10_min', 'objective_time_avg_per_10_min',
          'hero_damage_done_avg_per_10_min', 'healing_done_avg_per_10_min'
      ]

      # Campos adicionales de carrera para Ana
      additional_fields = [
          'scoped_accuracy_best_in_game', 'self_healing', 'self_healing_most_in_game',
          'enemies_slept', 'nano_boost_assists', 'unscoped_accuracy_best_in_game',
          'enemies_slept_most_in_game', 'nano_boost_assists_most_in_game',
          'biotic_grenade_kills'
      ]

      ana_stats = {}

      # Buscar los valores en heroes_comparisons
      heroes_comparisons = data['stats']['pc']['competitive']['heroes_comparisons']
      for attr in attributes:
        if attr in heroes_comparisons and heroes_comparisons[attr]:
            attr_values = heroes_comparisons[attr]['values']
            # Buscar el valor para el héroe 'ana'
            ana_data = next((item for item in attr_values if item['hero'] == 'ana'), None)
            ana_stats[attr] = ana_data['value'] if ana_data else None
        else:
            ana_stats[attr] = 0
            campos_cero=campos_cero+1

      # Buscar los valores en career_stats para Ana
      try:
        #print(player)
        if data['stats']['pc']['competitive']['career_stats']['ana']:
          career_stats = data['stats']['pc']['competitive']['career_stats'].get('ana', [{}])[0]
          # Extraer los campos adicionales
          for field in additional_fields:
              ana_stats[field] = next((stat['value'] for stat in career_stats.get('stats', []) if stat['key'] == field), None)
        else:
          #Si no hay datos específicos para el heroe se retorna None
          return None
      except KeyError as e:
          print(f"Error de clave en career_stats: {e}")
          #Si no hay datos específicos para el heroe se retorna None
          return None
      if campos_cero>10:
        #Si hay mas de 10 campos vacios de stats en general para el heroe se retorna None
        return None
      else:
        return ana_stats
    else:
      print(f"Perfil privado o datos no disponibles para {player}.")
      return None

# Lista de tiers a scrapear
tiers = ['bronze', 'silver', 'gold', 'platinum', 'diamond', 'master', 'grandmaster']
#tiers = ['diamond']

# Lista para almacenar todos los datos
all_data = []

# Iterar sobre cada tier y scrapear los datos
for tier in tiers:
    all_data.extend(scrape_tier_data(tier))

# Convertir la lista de datos en un DataFrame de pandas
df = pd.DataFrame(all_data, columns=['Player', 'Skill Tier'])

# Agregar las columnas para los atributos y los campos adicionales
attributes = [
    'time_played', 'games_won', 'win_percentage', 'weapon_accuracy_best_in_game',
    'eliminations_per_life', 'kill_streak_best', 'eliminations_avg_per_10_min',
    'deaths_avg_per_10_min', 'final_blows_avg_per_10_min', 'solo_kills_avg_per_10_min',
    'objective_kills_avg_per_10_min', 'objective_time_avg_per_10_min',
    'hero_damage_done_avg_per_10_min', 'healing_done_avg_per_10_min'
]
additional_fields = [
    'scoped_accuracy_best_in_game', 'self_healing', 'self_healing_most_in_game',
    'enemies_slept', 'nano_boost_assists', 'unscoped_accuracy_best_in_game',
    'enemies_slept_most_in_game', 'nano_boost_assists_most_in_game',
    'biotic_grenade_kills'
]

# Añadir columnas para los atributos y campos adicionales
all_columns = attributes + additional_fields
for column in all_columns:
    df[column] = None

# Iterar sobre cada jugador en el DataFrame y obtener sus estadísticas
for index, row in df.iterrows():
    player = row['Player']
    stats = get_player_stats(player)
    if stats is None:
      df.drop(index, inplace=True)
    else:
      for column, value in stats.items():
          if value is None:
            value=0
          if column in df.columns:
              df.at[index, column] = value

# Mostrar el DataFrame
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/overbuff_ana_rankings_with_full_stats.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Error al obtener datos para Waifu#21504: 404.
Error al obtener datos para Waifu#21504: 404.
Error al obtener datos para Waifu#21504: 404.
Perfil privado o datos no disponibles para Waifu#21504.
Error al obtener datos para GrandmaSauce#11440: 404.
Error al obtener datos para GrandmaSauce#11440: 404.
Error al obtener datos para GrandmaSauce#11440: 404.
Perfil privado o datos no disponibles para GrandmaSauce#11440.
Error al obtener datos para Sage534#1371: 404.
Error al obtener datos para Sage534#1371: 404.
Error al obtener datos para Sage534#1371: 404.
Perfil privado o datos no disponibles para Sage534#1371.
Error al obtener datos para Eros#13990: 404.
Error al obtener datos para Eros#13990: 404.
Error al obtener datos para Eros#13990: 404.
Perfil privado o datos no disponibles para Eros#13990.
Error de clave en career_stats: 'ana'
Error al obtener datos para R